Just importing stuff

In [4]:
# %pip install keras
# %pip install PyQt5
import pandas as pd
import numpy as np
import tensorflow as tf
import PyQt5
import matplotlib.pyplot as plt
import datetime as dt

ModuleNotFoundError: No module named 'PyQt5'

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score as acc_score
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize_scalar
from sklearn.metrics import mean_absolute_error

In [3]:
data = pd.read_csv("../complete_merged.csv",index_col=0)

Consecutive date checking function (Some districts had partial data)

Split Sequence(Standard function for splitting data for univariate lstm)

In [4]:
#Testing datetime consecutiveness


def checkConsecutiveDates(date_strs):
    dates = [dt.datetime.strptime(d, "%Y-%m-%d") for d in date_strs]

    date_ints = set([d.toordinal() for d in dates])
    if len(date_ints) == 1:
        return True
    elif max(date_ints) - min(date_ints) == len(date_ints) - 1:
        return True
    else:
        return False
    


def split_sequence(date_sequence,temp_sequence, n_steps,n_features):
    X, y = list(), list()
    for i in range(len(temp_sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
 # check if we are beyond the sequence
        if end_ix > len(temp_sequence)-1:
            break
 # gather input and output parts of the pattern
        seq_x, seq_y = temp_sequence[i:end_ix], temp_sequence[end_ix]
        date_seq = date_sequence[i:end_ix]
        # print(checkConsecutive(seq_x))
        if(checkConsecutiveDates(date_seq)):
            X.append(seq_x)
            y.append(seq_y)
    X,y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], n_features))
        
    return X,y

def get_data(district,column):
    return data[data['district'] == district].sort_values(axis = 0,by = "date")[column].to_list()
   



# date_strs = ['02-29-2012', '02-28-2012', '03-01-2012']
date_strs = ['1999-12-31','2000-01-01']
print(checkConsecutiveDates(date_strs))

X,y = split_sequence(['2021-01-09','2021-01-11','2021-01-12','2021-01-13','2021-01-14'],[10, 20, 30, 40, 50],3,1)
for i in range(len(X)):
    print(X[i], y[i])





True
[[20]
 [30]
 [40]] 50


Function to evaluate model(Mean Square Error)

In [5]:
def evalLSTM(model,X,y,n_steps,n_features):
    y_predict = list()
    for i in range(X.shape[0]):
        x_input = X[i]
        x_input = x_input.reshape((1, n_steps, n_features))
        predicted_value = model.predict(x_input, verbose=0)[0][0]
        print(len(y_predict),"of",X.shape[0])
        y_predict.append(predicted_value)
    
    mse = tf.keras.losses.MeanSquaredError()
    
    
    return {'mse':mse(y_predict,y).numpy(),'y_predict':y_predict}


Defining the model

In [112]:
#different optimizers and loss functions need to be tested yet
n_steps = 10
n_features = 1
epochs =200
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')



Basic check for Adilabad

In [113]:
%%script false 

data_Adilabad_temp = get_data('Adilabad','temp_max')
data_Adilabad_date = get_data('Adilabad','date')
# print(data_Adilabad_date)


X,y = split_sequence(data_Adilabad_date,data_Adilabad_temp,n_steps,n_features)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0., random_state=42)

model.fit(X_train, y_train, epochs=200, verbose=4)
# mse = evalLSTM(model,X,y,10,1)

# mse
#3.36 for 200 epochs


Couldn't find program: 'false'


In [114]:
mse = evalLSTM(model,X_test,y_test,10,1)

mse
r2 = acc_score(mse['y_predict'],y_test)
print(r2)

NameError: name 'X_test' is not defined

In [ ]:
# %%script false 
date_X,date_y = split_sequence(data_Adilabad_date,data_Adilabad_date,n_steps,n_features)

%matplotlib qt
date_y = np.asarray(date_y, dtype='datetime64[s]')

plt.plot(date_y,y)
plt.plot(date_y,mse['y_predict'])
plt.show()


Various window sizes were used on Adilabad for optimization.Results were as follows:

[10-3.289,
 20-55142.400,
 30-6766769.447,
 40-216242835.855,
 50-14841958357.522,
 60-793333303283.484]


[7-116.768,
 8-28.928,
 9-5.358,
 10-3.289,
 11-5.317]

Concluding that 10 is an optimal window size

Training on complete data of all districts and multiple window sizes


In [9]:

# %%script false


steps = [10,30]
# steps = [90]
for i in steps:
    n_steps = i
    n_features = 1
    epochs =200
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    j = 1
    for district in data['district'].unique():
        print(str(j) + "out of "+str(len(data['district'].unique())))
        
        data_unique_district_temp = get_data(district,'temp_max')
        data_unique_district_date = get_data(district,'date')
        
        print(district,str(len(data_unique_district_temp)))
        X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
        model.fit(X, y, epochs=epochs, verbose=0)
        j = j+1
        

    model.save('saved_model/model_e'+str(epochs)+'_w'+str(n_steps))

1out of 46
Adilabad 1733
2out of 46
Bhadradri-Kothagudem 1095
3out of 46
Hyderabad 1733
4out of 46
Jagtial 1733
5out of 46
Jangaon 1733
6out of 46
Jayashankar-Bhupalpally 1095
7out of 46
Jogulamba-Gadwal 1095
8out of 46
Kamareddy 1733
9out of 46
Karimnagar 1733
10out of 46
Khammam 1733
11out of 46
Kumuram Bheem - Asifabad 617
12out of 46
Mahabubabad 1733
13out of 46
Mahabubnagar 1733
14out of 46
Mancherial 1733
15out of 46
Medak 1733
16out of 46
Medchal-Malkajgiri 1733
17out of 46
Nagarkurnool 1733
18out of 46
Nalgonda 1733
19out of 46
Nirmal 1733
20out of 46
Nizamabad 1733
21out of 46
Peddapalli 1733
22out of 46
Rajanna-Siricilla 1095
23out of 46
Rangareddy 1733
24out of 46
Sangareddy 1733
25out of 46
Siddipet 1733
26out of 46
Suryapet 1733
27out of 46
Vikarabad 1733
28out of 46
Wanaparthy 1733
29out of 46
Warangal (R) 617
30out of 46
Warangal (U) 617
31out of 46
Yadadri-Bhongir 1095
32out of 46
Komaram Bheem-Asifabad 112
33out of 46
Mulug 213
34out of 46
Narayanpet 1217
35out of 46
W

Loading a model


In [118]:
model = tf.keras.models.load_model('./saved_model/my_model200')

n_steps = 10
n_features = 1


Comparing true and pred for adilabad

In [119]:
data_unique_district_temp = get_data('Adilabad','temp_max')
data_unique_district_date = get_data('Adilabad','date')
    
  
X,y = split_sequence(data_unique_district_date, data_unique_district_temp,10,n_features)

metrics10 = evalLSTM(model,X,y,10,1)




0 of 1714
1 of 1714
2 of 1714
3 of 1714
4 of 1714
5 of 1714
6 of 1714
7 of 1714
8 of 1714
9 of 1714
10 of 1714
11 of 1714
12 of 1714
13 of 1714
14 of 1714
15 of 1714
16 of 1714
17 of 1714
18 of 1714
19 of 1714
20 of 1714
21 of 1714
22 of 1714
23 of 1714
24 of 1714
25 of 1714
26 of 1714
27 of 1714
28 of 1714
29 of 1714
30 of 1714
31 of 1714
32 of 1714
33 of 1714
34 of 1714
35 of 1714
36 of 1714
37 of 1714
38 of 1714
39 of 1714
40 of 1714
41 of 1714
42 of 1714
43 of 1714
44 of 1714
45 of 1714
46 of 1714
47 of 1714
48 of 1714
49 of 1714
50 of 1714
51 of 1714
52 of 1714
53 of 1714
54 of 1714
55 of 1714
56 of 1714
57 of 1714
58 of 1714
59 of 1714
60 of 1714
61 of 1714
62 of 1714
63 of 1714
64 of 1714
65 of 1714
66 of 1714
67 of 1714
68 of 1714
69 of 1714
70 of 1714
71 of 1714
72 of 1714
73 of 1714
74 of 1714
75 of 1714
76 of 1714
77 of 1714
78 of 1714
79 of 1714
80 of 1714
81 of 1714
82 of 1714
83 of 1714
84 of 1714
85 of 1714
86 of 1714
87 of 1714
88 of 1714
89 of 1714
90 of 1714
91 of 171

In [129]:
date_X,date_y = split_sequence(data_unique_district_date,data_unique_district_date,n_steps,n_features)

%matplotlib qt
date_y = np.asarray(date_y, dtype='datetime64[s]')

plt.plot(date_y,y)
plt.plot(date_y,metrics10['y_predict'])
plt.show()
metrics10['y_predict']


[34.29557,
 31.838846,
 32.406647,
 33.636467,
 32.48134,
 32.604313,
 32.411644,
 31.203508,
 32.296207,
 31.23466,
 30.08022,
 30.833132,
 30.671747,
 31.604588,
 31.25205,
 30.393257,
 29.917803,
 29.997332,
 30.694824,
 31.612314,
 31.454617,
 31.68225,
 30.76711,
 32.78789,
 35.67689,
 36.365654,
 41.608192,
 37.255344,
 35.57909,
 33.507812,
 31.4867,
 34.056858,
 32.421967,
 31.857286,
 32.493732,
 31.101856,
 29.39108,
 30.724705,
 30.531923,
 32.834824,
 33.237465,
 33.471256,
 33.304195,
 32.752068,
 34.23275,
 34.15524,
 33.749775,
 34.586742,
 34.818695,
 35.150093,
 32.453842,
 33.87148,
 36.349815,
 36.144894,
 40.766518,
 36.318653,
 34.243626,
 32.888706,
 31.584429,
 34.215736,
 30.437914,
 31.072224,
 38.24833,
 35.672417,
 36.32138,
 34.93022,
 29.500729,
 35.755875,
 35.03852,
 35.571957,
 38.047863,
 36.35017,
 36.00125,
 36.76756,
 36.368244,
 36.49718,
 36.697456,
 38.699833,
 38.311485,
 38.253654,
 38.062542,
 34.32112,
 34.4607,
 36.817577,
 37.278713,
 39.842

Predicting future values using this basic model

In [136]:
model = tf.keras.models.load_model('saved_model/my_model200',compile=False)  

temp_2023 = np.array([29.0,28.0,27.0,21.0,19.0,26.0,27.0,28.0,30.0,31.0])

predicted_value = []
for i in range(100):
    temp_window = temp_2023[-10:]
    temp_window = temp_window.reshape(1, 10, n_features)
    final_pred = model.predict(temp_window, verbose=0)[0][0]       
    print(final_pred)
    temp_2023 = np.append(temp_2023, final_pred)
temp_2023
%matplotlib qt
plt.plot(temp_2023)
plt.show()

31.579882
30.712112
28.965204
28.257671
29.026407
28.8962
28.957882
28.584053
28.723291
28.402103
28.098099
27.41946
28.96599
29.696857
31.026712
31.446686
33.060574
33.515976
34.844166
34.737644
34.600548
33.466076
32.569225
31.713154
31.14247
30.556946
30.439747
30.219923
30.087114
29.856913
29.596376
29.24295
28.83124
28.203215
27.58487
27.102003
26.800102
26.70342
26.930246
27.584301
29.134857
31.29892
34.00701
36.45892
38.005398
38.5284
38.007336
36.624596
35.498837
34.704807
34.000412
33.626884
33.72821
35.32201
36.215023
34.948196
33.83106
33.09519
32.685158
32.37954
32.02296
31.492033
31.29195
31.268103
30.865633
30.467875
30.104465
29.724129
29.373878
28.926174
28.351263
27.861261
27.433535
26.960447
26.578484
26.484797
26.934612
28.156944
30.208612
32.867073
35.69159
37.830902
38.950314
39.069862
37.978813
36.536633
35.595814
34.88627
34.246883
34.141163
35.239815
36.26331
36.560387
35.121796
34.070915
33.410976
33.039757
32.936913
32.479866
32.13446


Comibing multiple predictions for ensembling

In [56]:

final_df = pd.DataFrame({'date':[],'actualTemp':[]})
steps = [5,10]
for step in steps:
    date_X,date_y = split_sequence(data_unique_district_date,data_unique_district_date,step,n_features)
    colname = 'temp'+str(step)
    X,y = split_sequence(data_unique_district_date, data_unique_district_temp,step,n_features)
    model = tf.keras.models.load_model('saved_model/model_e200_w'+str(step),compile=False)
    metrics = evalLSTM(model,X,y,step,1)
    temp_df = pd.DataFrame({'date':date_y,colname:metrics['y_predict'],'actualTemp':y})
    final_df = final_df.merge(temp_df, how='outer', on=['date','actualTemp']).sort_values(by=['date'])


final_processed = final_df.dropna(axis=0)


0 of 1724
1 of 1724
2 of 1724
3 of 1724
4 of 1724
5 of 1724
6 of 1724
7 of 1724
8 of 1724
9 of 1724
10 of 1724
11 of 1724
12 of 1724
13 of 1724
14 of 1724
15 of 1724
16 of 1724
17 of 1724
18 of 1724
19 of 1724
20 of 1724
21 of 1724
22 of 1724
23 of 1724
24 of 1724
25 of 1724
26 of 1724
27 of 1724
28 of 1724
29 of 1724
30 of 1724
31 of 1724
32 of 1724
33 of 1724
34 of 1724
35 of 1724
36 of 1724
37 of 1724
38 of 1724
39 of 1724
40 of 1724
41 of 1724
42 of 1724
43 of 1724
44 of 1724
45 of 1724
46 of 1724
47 of 1724
48 of 1724
49 of 1724
50 of 1724
51 of 1724
52 of 1724
53 of 1724
54 of 1724
55 of 1724
56 of 1724
57 of 1724
58 of 1724
59 of 1724
60 of 1724
61 of 1724
62 of 1724
63 of 1724
64 of 1724
65 of 1724
66 of 1724
67 of 1724
68 of 1724
69 of 1724
70 of 1724
71 of 1724
72 of 1724
73 of 1724
74 of 1724
75 of 1724
76 of 1724
77 of 1724
78 of 1724
79 of 1724
80 of 1724
81 of 1724
82 of 1724
83 of 1724
84 of 1724
85 of 1724
86 of 1724
87 of 1724
88 of 1724
89 of 1724
90 of 1724
91 of 172

Using scipy for finding weights that give minima

Local minimization


In [124]:
from scipy.optimize import minimize
from numpy.random import rand
 
# objective function
def objective_function(x):
    weighted_temp = x[0]*final_processed['temp5']+x[1]*final_processed['temp10']
    weighted_temp = weighted_temp.to_list()
    actual_temp = final_processed['actualTemp'].to_list()
    # print(len(weighted_temp),len(actual_temp))
    return mean_absolute_error(weighted_temp, actual_temp)
 
# define range for input
# r_min, r_max = -5.0, 5.0
# define the starting point as a random sample from the domain
pt = 1,1

# perform the l-bfgs-b algorithm search
result = minimize(objective_function,pt, method='L-BFGS-B')
# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution = result['x']
evaluation = objective_function(solution)
print('Solution: f(%s) = %.5f' % (solution, evaluation))

Status : CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Total Evaluations: 3
Solution: f([1. 1.]) = 33.76066


GLobal Minimization

In [96]:

# simulated annealing global optimization for a multimodal objective function
from scipy.optimize import dual_annealing
 
# objective function
def objective_function(x):
    weighted_temp = x[0]*final_processed['temp5']+x[1]*final_processed['temp10']
    weighted_temp = weighted_temp.to_list()
    actual_temp = final_processed['actualTemp'].to_list()
    # print(len(weighted_temp),len(actual_temp))
    return mean_absolute_error(weighted_temp, actual_temp)
 
# define range for input
r_min, r_max = 0.0, 1.0
# define the bounds on the search
bounds = [[r_min, r_max], [r_min, r_max]]
# perform the simulated annealing search
result = dual_annealing(objective_function, bounds)
# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution = result['x']
evaluation = objective_function(solution)
print('Solution: f(%s) = %.5f' % (solution, evaluation))

Status : ['Maximum number of iteration reached']
Total Evaluations: 4643
Solution: f([0.17933118 0.83370792]) = 1.67335


Status : ['Maximum number of iteration reached']
Total Evaluations: 4643
Solution: f([0.17933118 0.83370792]) = 1.67335

Now predicting future results

In [133]:
steps = [5,10]
# weights = [0.17933118,0.83370792]
weights = [1,0]
models = {}
for step in steps:
    models[str(step)] = tf.keras.models.load_model('saved_model/model_e200_w'+str(step),compile=False)  

temp_2023 = np.array([29.0,28.0,27.0,21.0,19.0,26.0,27.0,28.0,30.0,31.0])
predicted_value = []
for i in range(100):
    final_pred = 0
    predicted_value = []
    for step in steps:
        temp_window = temp_2023[-1*int(step):]
        temp_window = temp_window.reshape(1, step, n_features)
        step_pred = models[str(step)].predict(temp_window, verbose=0)[0][0]
        print(step,step_pred)
        predicted_value= np.append(predicted_value, step_pred)
        
    print(predicted_value)
    final_pred = np.sum(np.multiply(weights , predicted_value))
    print(final_pred)
    temp_2023 = np.append(temp_2023, final_pred)
    

5 28.717566
10 31.378061
[28.71756554 31.37806129]
28.717565536499023
5 28.88946
10 29.79211
[28.88945961 29.79211044]
28.88945960998535
5 27.960308
10 30.389986
[27.96030807 30.38998604]
27.960308074951172
5 27.538288
10 36.8955
[27.53828812 36.89550018]
27.538288116455078
5 30.55862
10 35.018005
[30.55862045 35.01800537]
30.55862045288086
5 35.82961
10 31.859255
[35.82960892 31.85925484]
35.82960891723633
5 34.890244
10 34.096947
[34.89024353 34.09694672]
34.89024353027344
5 27.40658
10 34.008095
[27.40657997 34.00809479]
27.406579971313477
5 26.486816
10 30.710508
[26.48681641 30.71050835]
26.48681640625
5 28.624939
10 29.234018
[28.62493896 29.23401833]
28.62493896484375
5 32.935104
10 28.154236
[32.93510437 28.15423584]
32.93510437011719
5 30.559631
10 33.6849
[30.55963135 33.68489838]
30.55963134765625
5 28.721601
10 30.427343
[28.72160149 30.42734337]
28.721601486206055
5 30.42798
10 30.400434
[30.42798042 30.40043449]
30.427980422973633
5 32.85685
10 31.270535
[32.85684967 31.2

In [135]:
%matplotlib qt
# date_y = np.asarray(date_y, dtype='datetime64[s]')

# plt.plot(date_y,y)
plt.plot(temp_2023)
plt.show()

In [ ]:
# %matplotlib qt
date_y = np.asarray(date_y, dtype='datetime64[s]')

plt.plot(date_y,y)
plt.plot(date_y,metrics['y_predict'])
plt.show()

In [10]:
# %%script false

temp_2023 = np.array([29.0,28.0,27.0,21.0,19.0,26.0,27.0,28.0,30.0,31.0])
temp_window = temp_2023[-10:]
for i in range(100):
    temp_window = temp_window.reshape(1, n_steps, n_features)
    predicted_value = model.predict(temp_window, verbose=0)[0][0]
    print(predicted_value)
    temp_2023 = np.append(temp_2023, predicted_value)
    temp_window = temp_2023[-10:]


31.579882
30.712112
28.965204
28.257671
29.026407
28.8962
28.957882
28.584053
28.723291
28.402103
28.098099
27.41946
28.96599
29.696857
31.026712
31.446686
33.060574
33.515976
34.844166
34.737644
34.600548
33.466076
32.569225
31.713154
31.14247
30.556946
30.439747
30.219923
30.087114
29.856913
29.596376
29.24295
28.83124
28.203215
27.58487
27.102003
26.800102
26.70342
26.930246
27.584301
29.134857
31.29892
34.00701
36.45892
38.005398
38.5284
38.007336
36.624596
35.498837
34.704807
34.000412
33.626884
33.72821
35.32201
36.215023
34.948196
33.83106
33.09519
32.685158
32.37954
32.02296
31.492033
31.29195
31.268103
30.865633
30.467875
30.104465
29.724129
29.373878
28.926174
28.351263
27.861261
27.433535
26.960447
26.578484
26.484797
26.934612
28.156944
30.208612
32.867073
35.69159
37.830902
38.950314
39.069862
37.978813
36.536633
35.595814
34.88627
34.246883
34.141163
35.239815
36.26331
36.560387
35.121796
34.070915
33.410976
33.039757
32.936913
32.479866
32.13446


In [ ]:

%matplotlib qt
# date_y = np.asarray(date_y, dtype='datetime64[s]')

# plt.plot(date_y,y)
plt.plot(temp_2023)
plt.show()

Trying to create a csv file(cell in progress)


In [ ]:
%%script false

final_df = pd.DataFrame()



i = 1
# for district in data['district'].unique():
for district in ['Adilabad']:
    temp_df = pd.DataFrame()
    print(str(i) + "out of "+str(len(data['district'].unique())))
    
    data_unique_district_temp = get_data(district,'temp_max')
    data_unique_district_date = get_data(district,'date')
   
    print(district,str(len(data_unique_district_temp)))
    X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
    date_X,date_y = split_sequence(data_unique_district_date,data_unique_district_date,n_steps,n_features)
    metrics = evalLSTM(model,X,y,10,1)

    temp_df['date'] = date_y
    temp_df['predict_temp_max'] = metrics['y_predict']
    temp_df['district'] = district
    temp_df['window_size'] = 10

    final_df.concat(temp_df)
    
    
    i = i+1

temp_df